In [18]:
import scipy.special
import shap
import numpy as np
import scipy
import itertools
import pandas as pd

データセットのダウンロード

In [19]:
dataset_loaders = {
    "Adult": shap.datasets.adult,
    "California": shap.datasets.california,
    "Communities": shap.datasets.communitiesandcrime,
    "Correlated": shap.datasets.corrgroups60,
    "Diabetes": shap.datasets.diabetes,
    "Independent": shap.datasets.independentlinear60,
    "IRIS": shap.datasets.iris,
    "NHANES": shap.datasets.nhanesi,
}

In [20]:
def load_dataset(dataset_name):
    X, y = dataset_loaders[dataset_name]()
    # Remove nan values
    X = X.fillna(X.mean())
    return X, y


In [21]:
import xgboost as xgb

X, y = load_dataset("Adult") # すべてのデータセットで動いた
n = X.shape[1] # 特徴量数
# Assuming deterministic
model = xgb.XGBRegressor(n_estimators=100, max_depth=4)
model.fit(X, y) # 説明対象モデルを訓練

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [22]:
# X:データセットの特徴量(load_datasetで取得したX)
def load_input(X, seed=None, is_synthetic=False):
    if is_synthetic:
        baseline = np.zeros((1, X.shape[1]))  # すべての特徴量が0（存在しない）のベースライン
        explicand = np.ones((1, X.shape[1]))  # すべての特徴量が1（存在する存在する）の説明対象
        return baseline, explicand
    if seed is not None:
        np.random.seed(seed)
    baseline = X.mean().values.reshape(
        1, -1
    )  # 各特徴量の平均値をベースラインとして使用。.reshape(1, -1)で1行n列 の2次元配列に整形
    explicand_idx = np.random.choice(X.shape[0])  # 説明対象のインデックスをランダムに選択
    explicand = X.iloc[explicand_idx].values.reshape(1, -1)  # 説明対象の行の特徴量を取得
    for i in range(explicand.shape[1]):  # 説明対象のインスタンスの各特徴量について
        # baseline と explicandが各特徴量ごとに必ず異なる値を持つよう保証する
        # SHAP 等の説明手法では、ベースラインとの差分を計算するため、ある特徴量でベースラインと同値だと説明が得られない（差分が 0）というケースを避けたい、という意図がある
        while baseline[0, i] == explicand[0, i]:
            explicand_idx = np.random.choice(X.shape[0])
            explicand[0, i] = X.iloc[explicand_idx, i]
    return baseline, explicand


1インスタンス分の説明を生成

In [23]:
# 説明の計算のためにベースラインと説明対象を取得
baseline, explicand = load_input(X)

In [ ]:
is_small = 2**n <= 1e7

In [ ]:
# Compute the true SHAP values (assuming tree model)
def official_tree_shap(baseline, explicand, model, num_samples):
    # Suppress warning only for this function
    import warnings

    warnings.filterwarnings("ignore", category=UserWarning)
    explainer = shap.TreeExplainer(model, baseline)
    shap_values = explainer.shap_values(explicand)
    # Re-enable warnings
    warnings.filterwarnings("default", category=UserWarning)
    return shap_values

In [ ]:
# コード内では以下のように呼び出す

# Compute the true SHAP values (assuming tree model)
# true_shap_values = estimators['Official Tree SHAP'](baseline, explicand, model, sample_size).flatten()

# leverageshap\estimators\__init__.pyにて以下のように定義されているためこれが可能
# estimators = {
#     #    'Monte Carlo': monte_carlo,
#     #    'Matrix SHAP': matrix_shap,
#     #    'Matrix SHAP wo Bernoulli': matrix_shap_wo_bernoulli,
#     #    'Matrix SHAP wo Bernoulli, Paired': matrix_shap_wo_bernoulli_paired,
#     "Kernel SHAP": kernel_shap,
#     "Optimized Kernel SHAP": official_kernel_shap,
#     "Leverage SHAP": leverage_shap,
#     "Kernel SHAP Paired": kernel_shap_paired,
#     #    'Leverage SHAP wo Bernoulli': leverage_shap_wo_bernoulli,
#     #    'Leverage SHAP wo Bernoulli, Paired': leverage_shap_wo_bernoulli_paired,
#     "Official Tree SHAP": official_tree_shap,
#     "Leverage SHAP (Unpaired)": leverage_shap_wo_paired,
#     "Permutation SHAP": official_permutation_shap,
# }

In [ ]:
true_shap_values = official_tree_shap(baseline, explicand, model, 5*n).flatten()

In [31]:
true_shap_values


array([-0.00968549, -0.00302589,  0.        , -0.00662473, -0.00045752,
       -0.00323403,  0.        ,  0.01480636,  0.00612298, -0.05661553,
        0.        ,  0.01550115])

In [29]:
small_setup = {}
